本次我们从公交网获取北京公交的数据。

(http://beijing.gongjiao.com/lines_all.html)

![](./img/数据获取基本流程.png)

# 如何用Python模拟网络请求

使用request库可以模拟不同的请求，例如requests.get()模拟get请求，requests.post()模拟post请求。必要的时候可以添加请求头header，header通常包括user-agent，cookie，refer等信息，还可以增加请求参数data和代理信息。主要代码形式为：response = requests.request("GET", url, headers=headers, params=querystring)response是网站返回的响应信息，可以调用其text方法获取网站的HTML源码

In [8]:
import requests
import lxml
from lxml import etree
import pandas as pd

In [3]:
url = 'http://beijing.gongjiao.com/lines_all.html'
text = requests.get(url).text

# 如何对网页进行解析

python中提供了多种库用于网页解析，例如lxml，BeautifulSoup，pyquery等。每一个工具都有相应的解析规则，但都是把HTML文档当做一个DOM树，通过选择器进行节点和属性的定位。

In [10]:
doc = etree.HTML(text)
all_lines = doc.xpath("//div[@calss='list']/ul/li")
for line in all_lines:
    line_name = line.xpath("./a/text()")[0].strip()
    line_url = line.xpath("./a/@href")[0]

第一行代码将上一步返回的HTML文本转换为xpath可以解析的对象。第二行代码定位到class=list的div下面所有的li标签，即右图中的红色框的部分，得到的是一个列表。从第三行开始对其进行遍历，处理每一个li下面的a标签。第4行取出a标签下的文本，用到了xpath的text()方法，对应到第一个li就是“北京1路公交车路线”，第5行取出a标签下对应的链接，用到了xpath的@href取出a标签下的href属性值。直接取都是列表的形式，所以需要用索引取出具体的值。

这样我们就可以得到整个公交线路列表中的线路名称和线路url。然后从线路url出发，就可以获取每条线路的具体信息。如下面代码和图片所示，虽然数据略多，但主要的逻辑和上面类似.

# 如何存储获取的数据

In [13]:
#准备一个存储数据的字典
df_dict = {
    'line_name': [],
    'line_url': [],
    'line_start': [],
    'line_stop': [],
    'line_op_time': [],
    'line_interval': [],
    'line_price': [],
    'line_company': [],
    'line_up_times': [],
    'line_station_up': [],
    'line_station_up_len': [],
    'line_station_down': [],
    'line_station_down_len': []
}

数据存储的载体通常有文件(例如csv，excel)和数据库(例如mysql，MongoDB)。我们这里选择了csv文件的形式，一方面是数据量不是太大，另一方面也不需要进行数据库安装，只需将数据整理成dataframe的格式，直接调用pandas的to_csv方法就可以将dataframe写入csv文件中。

以上我们分模拟请求，网页解析，数据存储3个步骤，学习了数据获取的流程。实际运行过程中，还需要增加一些保证代码“健壮性”的逻辑。例如，控制爬取的频率，处理请求失败的情况，处理不同的线路网页结构可能有差异的情况等等。本次的数据源没有做很多反扒限制，因此前两种情况我们可以不处理。至于第三种，有的路线会出现线路运营时间是空值的情况，需要进行判断。另外还可以增加一些爬虫运行过程的提示信息，让我们知道爬取进度，当然你也可以增加多线程，代理，ua切换等代码，此处我们还用不上这些。

In [17]:
# # 数据获取完成代码
# import requests
# import pandas as pd
# from lxml import etree
# url = 'http://beijing.gongjiao.com/lines_all.html'
# text = requests.get(url).text
# doc = etree.HTML(text)
# all_lines = doc.xpath("//div[@class='list']/ul/li")

# df_dict = {
#     'line_name': [],
#     'line_url': [],
#     'line_start': [],
#     'line_stop': [],
#     'line_op_time': [],
#     'line_interval': [],
#     'line_price': [],
#     'line_company': [],
#     'line_up_times': [],
#     'line_station_up': [],
#     'line_station_up_len': [],
#     'line_station_down': [],
#     'line_station_down_len': []
# }

# for line in all_lines:
#     line_name = line.xpath("./a/text()")[0].strip()
#     line_url = line.xpath("./a/@href")[0]
#     info_text=requests.get(line_url).text
#     info_doc=etree.HTML(info_text)
#     infos=info_doc.xpath("//div[@class='gj01_line_header clearfix']")
#     for info in infos:
#         start_stop=info.xpath("./dl/dt/a/text()")
#         op_times=info.xpath("./dl/dd[1]/b/text()")
#         if len(op_times)==0:
#             op_times=["未知"]
#         interval=info.xpath("./dl/dd[2]/text()")
#         if len(interval)==0:
#             interval=["未知"]
#         price=info.xpath("./dl/dd[3]/text()")
#         if len(price)==0:
#             price=["未知"]
#         company = info.xpath("./dl/dd[4]/text()")
#         if len(company) == 0:
#             company = ["未知"]
#         up_times = info.xpath("./dl/dd[5]/text()")
#         if len(up_times) == 0:
#             up_times = ["未知"]
#         all_stations_up = info_doc.xpath('//ul[@class="gj01_line_img JS-up clearfix"]')
#         for station in all_stations_up:
#             station_up_name = station.xpath('./li/a/text()')
#             if len(station_up_name) == 0:
#                 station_up_name = ["未知"]
#         all_stations_down = info_doc.xpath('//ul[@class="gj01_line_img JS-down clearfix"]')
#         for station in all_stations_down:
#             station_down_name = station.xpath('./li/a/text()')
#             if len(station_down_name) == 0:
#                 station_down_name = ["未知"]
#     try:
#         df_dict['line_name'].append(line_name)
#         df_dict['line_url'].append(line_url)
#         df_dict['line_start'].append(start_stop[0])
#         df_dict['line_stop'].append(start_stop[1])
#         df_dict['line_op_time'].append(op_times[0])
#         df_dict['line_interval'].append(interval[0][5:])
#         df_dict['line_company'].append(company[0][5:])
#         df_dict['line_price'].append(price[0][5:])
#         df_dict['line_up_times'].append(up_times[0][5:])
#         df_dict['line_station_up'].append(station_up_name)
#         df_dict['line_station_up_len'].append(len(station_up_name))
#         df_dict['line_station_down'].append(station_down_name)
#         df_dict['line_station_down_len'].append(len(station_down_name))
#         print('{} success!'.format(line_name))
#     except:
#         print(line_name, line_url)
#         continue

# df = pd.DataFrame(df_dict)
# df.to_csv('bjgj_lines_utf8.csv', index=None)
# print("OK！爬取完毕")

北京1路公交车路线 success!
北京2路公交车路线 success!
北京5路公交车路线 success!
北京6路公交车路线 success!
北京7路公交车路线 success!
北京8路公交车路线 success!
北京9路公交车路线 success!
北京10路公交车路线 success!
北京11路公交车路线 success!
北京12环行公交车路线 success!
北京13路公交车路线 success!
北京14路公交车路线 success!
北京15路公交车路线 success!
北京16路公交车路线 success!
北京17路公交车路线 success!
北京18路公交车路线 success!
北京19路公交车路线 success!
北京20路公交车路线 success!
北京21路公交车路线 success!
北京22路公交车路线 success!
北京22路公交车路线 success!
北京23路公交车路线 success!
北京24路公交车路线 success!
北京25路公交车路线 success!
北京26路公交车路线 success!
北京27路公交车路线 success!
北京28路公交车路线 success!
北京29路公交车路线 success!
北京30路公交车路线 success!
北京31路公交车路线 success!
北京32路公交车路线 success!
北京33路公交车路线 success!
北京34路公交车路线 success!
北京35路公交车路线 success!
北京36环行公交车路线 success!
北京37路公交车路线 success!
北京38路公交车路线 success!
北京39路公交车路线 success!
北京40路公交车路线 success!
北京41路公交车路线 success!
北京42路公交车路线 success!
北京43路公交车路线 success!
北京44内环公交车路线 success!
北京44外环公交车路线 success!
北京45路公交车路线 success!
北京46路公交车路线 success!
北京47路公交车路线 success!
北京48路公交车路线 success!
北京49路公交车路线 success!
北京50路公交车路线 success!
北京5

北京535路公交车路线 success!
北京536路公交车路线 success!
北京537路公交车路线 success!
北京538路公交车路线 success!
北京539路公交车路线 success!
北京540路公交车路线 success!
北京541路公交车路线 success!
北京542路公交车路线 success!
北京543路公交车路线 success!
北京544路公交车路线 success!
北京545路公交车路线 success!
北京546路公交车路线 success!
北京547路公交车路线 success!
北京548路公交车路线 success!
北京550路公交车路线 success!
北京551路公交车路线 success!
北京552路公交车路线 success!
北京553路公交车路线 success!
北京554路公交车路线 success!
北京555路公交车路线 success!
北京556路公交车路线 success!
北京557路公交车路线 success!
北京558路公交车路线 success!
北京559路公交车路线 success!
北京560路公交车路线 success!
北京561路公交车路线 success!
北京562路公交车路线 success!
北京563区间公交车路线 success!
北京563路公交车路线 success!
北京564路公交车路线 success!
北京565路公交车路线 success!
北京566路公交车路线 success!
北京567路公交车路线 success!
北京568路公交车路线 success!
北京569路公交车路线 success!
北京570路公交车路线 success!
北京571路公交车路线 success!
北京572路公交车路线 success!
北京573路公交车路线 success!
北京574路公交车路线 success!
北京575路公交车路线 success!
北京576路公交车路线 success!
北京578路公交车路线 success!
北京579路公交车路线 success!
北京580路公交车路线 success!
北京581路公交车路线 success!
北京582路公交车路线 success!
北京583路公交车路线 

北京948路公交车路线 success!
北京950路公交车路线 success!
北京951路公交车路线 success!
北京951区间公交车路线 success!
北京952路公交车路线 success!
北京953路(2014年1月29日起临时绕行)公交车路线 success!
北京954路公交车路线 success!
北京955路公交车路线 success!
北京957路公交车路线 success!
北京958路公交车路线 success!
北京959路公交车路线 success!
北京960路公交车路线 success!
北京961路公交车路线 success!
北京962路公交车路线 success!
北京963路公交车路线 success!
北京964路公交车路线 success!
北京965路公交车路线 success!
北京966路公交车路线 success!
北京966区间公交车路线 success!
北京967路公交车路线 success!
北京968路公交车路线 success!
北京969路公交车路线 success!
北京970路公交车路线 success!
北京971路公交车路线 success!
北京972路公交车路线 success!
北京973路公交车路线 success!
北京974路公交车路线 success!
北京975路公交车路线 success!
北京976路公交车路线 success!
北京977快车定班公交车路线 success!
北京977路公交车路线 success!
北京977快车公交车路线 success!
北京978路公交车路线 success!
北京978快车公交车路线 success!
北京980路公交车路线 success!
北京980快车公交车路线 success!
北京981路公交车路线 success!
北京982路公交车路线 success!
北京983路公交车路线 success!
北京984路公交车路线 success!
北京985区间公交车路线 success!
北京985路公交车路线 success!
北京986路公交车路线 success!
北京987路公交车路线 success!
北京988路[南段]公交车路线 success!
北京988路[北段]公交车路线 success!


北京顺13路公交车路线 success!
北京H13路区间莲花池公交车路线 success!
北京Y13支线公交车路线 success!
北京昌13路公交车路线 success!
北京M13路[煤窝]公交车路线 success!
北京密13路公交车路线 success!
北京平13路公交车路线 success!
北京H13路公交车路线 success!
北京房14区间[东港]公交车路线 success!
北京房14路[陈家坟]公交车路线 success!
北京兴14路公交车路线 success!
北京H14路公交车路线 success!
北京房14路公交车路线 success!
北京顺14路公交车路线 success!
北京平14路公交车路线 success!
北京Y14路公交车路线 success!
北京M14路[张家铺]公交车路线 success!
北京Y14支1路公交车路线 success!
北京Y14支2路公交车路线 success!
北京Y14支3路公交车路线 success!
北京通14路公交车路线 success!
北京通15路公交车路线 success!
北京平15路公交车路线 success!
北京兴15路公交车路线 success!
北京顺15路公交车路线 success!
北京房15路公交车路线 success!
北京密15路公交车路线 success!
北京Y15路公交车路线 success!
北京H15区间公交车路线 success!
北京M15路[爨底下]公交车路线 success!
北京H15路公交车路线 success!
北京昌16路公交车路线 success!
北京M16路[向阳口]公交车路线 success!
北京Y16路公交车路线 success!
北京兴16路公交车路线 success!
北京通16路公交车路线 success!
北京平16路公交车路线 success!
北京H16路公交车路线 success!
北京房16路公交车路线 success!
北京兴16区间公交车路线 success!
北京密16路公交车路线 success!
北京顺16路公交车路线 success!
北京H17支线公交车路线 success!
北京Y17路公交车路线 success!
北京M17路[大村]公交车路线 success!
北京H17路公

北京917野三坡旅游专线(跨省)公交车路线 success!
北京918盘山旅游专线(跨省)公交车路线 success!
北京980金山岭旅游专线(跨省)公交车路线 success!
北京黄花城水长城一日游公交车路线 success!
北京龙庆峡山水精华一日游公交车路线 success!
北京慕田峪长城精华一日游公交车路线 success!
北京前门大街有轨电车游览线公交车路线 success!
北京清东陵文化遗产一日游公交车路线 success!
北京神堂峪-雁栖湖一日游公交车路线 success!
北京十渡-孤山寨一日游公交车路线 success!
北京石林峡-金海湖一日游公交车路线 success!
北京世界文化遗产游A线公交车路线 success!
北京世界文化遗产游B线公交车路线 success!
北京世界文化遗产游C线公交车路线 success!
北京市内观光游精选二线公交车路线 success!
北京市内观光游精选三线公交车路线 success!
北京市内观光游精选一线公交车路线 success!
北京松山原始森林公园精品游公交车路线 success!
北京天津盘山一日游公交车路线 success!
北京长河段航线公交车路线 success!
北京昆玉段航线公交车路线 success!
北京南护段航线公交车路线 success!
北京转河段航线公交车路线 success!
北京956路(2014年2月26日起运行)公交车路线 success!
北京专69路(2014年2月26日起运营 原450路)公交车路线 success!
北京夜1路公交车路线 success!
北京夜2路公交车路线 success!
北京夜3路公交车路线 success!
北京夜4路公交车路线 success!
北京夜5路公交车路线 success!
北京夜6路公交车路线 success!
北京夜7路公交车路线 success!
北京夜8路公交车路线 success!
北京夜9路公交车路线 success!
北京夜10内公交车路线 success!
北京夜10外公交车路线 success!
北京夜11路公交车路线 success!
北京夜12路公交车路线 success!
北京夜13路公交车路线 success!
北京夜14路公交车路线 success!
北京夜15路公交车路线 s

北京专161环行电动车公交车路线 success!
北京专162环行电动车公交车路线 success!
北京专163电动车公交车路线 success!
北京专164电动车公交车路线 success!
北京专165电动车公交车路线 success!
北京专166电动车公交车路线 success!
北京专167电动车公交车路线 success!
北京专169电动车公交车路线 success!
北京城际班车[丰宁](跨省)公交车路线 success!
北京城际通勤班车[潮白河孔雀城-地铁潞城站](跨省) success!
北京顺义木林1路公交车路线 success!
北京开发区1外环公交车路线 success!
北京Y07支线公交车路线 success!
北京Y10内环公交车路线 success!
北京兴14区间公交车路线 success!
北京顺18区间公交车路线 success!
北京通19专线公交车路线 success!
北京兴21延长线公交车路线 success!
北京Y29路公交车路线 success!
北京顺30区间公交车路线 success!
北京Y32路公交车路线 success!
北京顺39区间公交车路线 success!
北京兴45路公交车路线 success!
北京顺46路公交车路线 success!
北京兴46路公交车路线 success!
北京兴52路公交车路线 success!
北京兴54路公交车路线 success!
北京兴55路公交车路线 success!
北京兴56路公交车路线 success!
北京兴59路公交车路线 success!
北京大兴北工大软件园-地铁荣京东街站接驳线 success!
北京大兴国际企业大道-地铁经海路站接驳线 success!
北京大兴马赛公馆-地铁枣园站接驳线 success!
北京大兴清河园-地铁旧宫站接驳线 success!
北京大兴数字工厂-地铁经海路站接驳线 success!
北京大兴兴康家园-地铁枣园站接驳线 success!
北京机场15线[世纪坛]公交车路线 success!
北京机场16线[石景山线]公交车路线 success!
北京0301次[管庄-国贸]公交车路线 success!
北京0501次[西红门-国贸]公交车路线 success!
北京1401次[果园-国贸]公交车路线 suc

# 如何读取数据

In [19]:
ori_data = pd.read_csv('./data/bjgj_lines_utf8.csv')
ori_data.head()

,line_name,line_url,line_start,line_stop,line_op_time,line_interval,line_price,line_company,line_up_times,line_station_up,line_station_up_len,line_station_down,line_station_down_len
0,北京1路公交车路线,http://beijing.gongjiao.com/xianlu_22546,老山公交场站(1),四惠枢纽站(27),5:00-23:00,未知,10公里以内票价2元，每增加5公里以内加价1元，最高票价6元,北京公交集团第六客运分公司,2015-04-05 03:32:16,"['老山公交场站(1)', '老山南路东口(2)', '地铁八宝山站(3)', '玉泉路口西...",23,"['四惠枢纽站(27)', '八王坟西(24)', '郎家园(23)', '大北窑东(23)...",24
1,北京2路公交车路线,http://beijing.gongjiao.com/xianlu_22547,海户屯(1),宽街路口南(12),5:00-23:00,未知,10公里以内票价2元，每增加5公里以内加价1元，最高票价3元,北京公交集团第二客运分公司,2015-04-05 03:32:17,"['海户屯(1)', '木樨园桥北(3)', '沙子口(3)', '永定门内(4)', '天...",12,"['宽街路口南(12)', '沙滩路口南(10)', '北京妇产医院(10)', '东华门(...",12
2,北京5路公交车路线,http://beijing.gongjiao.com/xianlu_22548,菜户营桥(1),北土城公交场站(24),5:10-22:30,未知,无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价5元,北京公交集团第二客运分公司,2015-04-05 03:32:17,"['菜户营桥(1)', '菜户营桥东(2)', '菜户营桥北(2)', '白纸坊桥北(3)'...",37,"['北土城公交场站(20)', '地铁北土城站(19)', '裕民中路(18)', '裕民中...",34
3,北京6路公交车路线,http://beijing.gongjiao.com/xianlu_22549,龙潭公园(1),六里桥东(12),5:00-24:00,未知,无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价3元,北京公交集团电车客运分公司,2015-04-05 03:32:18,"['龙潭公园(1)', '北京体育馆(2)', '北京体育馆西(3)', '法华寺(3)',...",21,"['六里桥东(12)', '莲花池(12)', '湾子(12)', '广外甘石桥(11)',...",21
4,北京7路公交车路线,http://beijing.gongjiao.com/xianlu_22550,五间楼,动物园枢纽站,5:00-23:00,未知,无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价4元,北京公交集团第二客运分公司,2015-04-05 03:32:19,"['五间楼', '贾家花园', '赵公口桥南', '定安路', '琉璃井', '安乐林', ...",28,"['动物园枢纽站', '西直门外', '西直门内', '新开胡同', '宝产胡同', '祖家...",30


# 如何进行数据预处理

## 找出重复数据

In [22]:
# 查看每一列数据的唯一值个数
ori_data.nunique()

line_name                1986
line_url                 2002
line_start                991
line_stop                1124
line_op_time              560
line_interval               4
line_price                126
line_company               82
line_up_times             650
line_station_up          1930
line_station_up_len        80
line_station_down        1701
line_station_down_len      80
dtype: int64

由于线路很多，我们在原始网页中很难发现是否会有重复的线路。但从上面观察line_name和line_url两个字段，line_name有1986个唯一值，line_url有2002个唯一值。说明line_name存在重复：会有名称相同的线路对应不同的line_url。所以接下来我们需要进行重复值的剔除。

出现了线路名称的重复，但却有不同的line_url，究竟是确实是线路“重名”还是线路“重复”？我们需要看一下数据重复的具体情况。因此需要把重复的行都找出来看看。可以使用pandas的duplicated方法，它可以对dataframe的指定列查看是否重复，返回True和False

In [23]:
d = ori_data.duplicated(subset=['line_name'])
d

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20       True
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1972    False
1973    False
1974    False
1975    False
1976    False
1977    False
1978    False
1979    False
1980    False
1981    False
1982    False
1983     True
1984     True
1985     True
1986     True
1987     True
1988     True
1989     True
1990     True
1991     True
1992    False
1993    False
1994    False
1995    False
1996    False
1997    False
1998    False
1999    False
2000    False
2001    False
Length: 2002, dtype: bool

In [24]:
# 显示重复数据
dup_data = ori_data[d]
dup_data

,line_name,line_url,line_start,line_stop,line_op_time,line_interval,line_price,line_company,line_up_times,line_station_up,line_station_up_len,line_station_down,line_station_down_len
20,北京22路公交车路线,http://beijing.gongjiao.com/xianlu_22566,牡丹园西,前门,5:00-24:00,未知,无人售票线路，单一票制1元，不设找赎。持卡乘车普通卡0.40元/次、学生卡0.20元/次,北京公交集团电车客运分公司,2015-01-29 07:33:25,"['牡丹园西', '牡丹园', '北太平桥南', '铁狮子坟', '北京师范大学', '小西...",19,"['前门', '前门西', '和平门东', '宣武门东', '西单路口南', '西单商场',...",18
279,北京422路公交车路线,http://beijing.gongjiao.com/xianlu_22825,三义庙,来广营西桥东,5:30-21:00,未知,12公里以内票价1元，每增加5公里以内加价0.5元。持卡乘车普通卡0.40元/次、学生卡0....,北京公交集团第八客运分公司,2015-01-29 07:33:23,"['三义庙', '四通桥东', '红民村', '大钟寺', '蓟门桥西', '蓟门桥东', ...",29,"['来广营西桥东', '来广营路口西', '利泽中街西口', '望京花园西区', '湖光北街...",29
1704,北京通11路公交车路线,http://beijing.gongjiao.com/xianlu_41275,刘庄,董村,05:40-18:30,未知,最高票价6元,北京恒基客运有限公司,2015-01-29 07:30:32,"['刘庄', '刘庄北口', '范庄', '结研所', '北京财贸职业学院', '八里桥市场...",48,"['董村', '北神树东口', '北神树西口', '北神树小学', '次渠银行', '科创二...",46
1705,北京通45路公交车路线,http://beijing.gongjiao.com/xianlu_41317,通州三中,通州中医院站,06:00-19:00,10分钟,刷卡优惠成人4折/学生2折,未知,2015-01-29 07:30:32,"['通州三中', '通州古城村南口', '新安屯村', '荔景园小区', '玉带河东街路口南...",19,"['通州中医院站', '翠屏小区东站', '果园环岛南', '城铁九棵树站', '城铁九棵树...",19
1728,北京顺45路公交车路线,http://beijing.gongjiao.com/xianlu_42373,西门,木林,7:29-18:05,未知,NaN,未知,2015-01-29 07:30:23,"['西门', '谊宾商城', '顺义火车站', '顺鑫农业', '隆华', '地铁顺义站',...",38,"['木林', '陈各庄', '蒋各庄', '荣各庄', '后王各庄', '前王各庄', '李...",38
1744,北京大红门-槐房西路直达快车公交车路线,http://beijing.gongjiao.com/xianlu_42839,南苑路果园,槐房路南口,16:40-17:00,未知,1元,未知,2015-01-29 07:30:19,"['南苑路果园', '久敬庄', '和义南站', '三营门', '红房子路口东', '槐房路...",6,"['丽园路公交站', '丽园路西口', '枣园路西口', '大兴八中', '骏城小区', '...",33
1752,北京686路公交车路线,http://beijing.gongjiao.com/xianlu_43029,建欣苑,朝新嘉园,5:30-22:00,未知,单程最高2.5元,未知,2015-01-29 07:30:17,"['建欣苑', '建欣苑中街', '大红门', '南苑路果园', '木樨园桥东', '赵公口...",31,"['朝新嘉园', '朝阳新城', '高杨树', '平房东口', '平房', '姚家园西', ...",30
1983,北京922路公交车路线,http://beijing.gongjiao.com/xianlu_47005,朝凤庵村,地铁天通苑北站,5:30-19:30,未知,10公里以内票价2元，每增加5公里以内加价1元，最高票价8元,北京八方达客运有限责任公司怀柔分公司(服务热线:010-69626820),2015-08-04 05:35:13,"['朝凤庵村', '昌平东关', '中国政法大学(昌平)', '燕平家园', '东环北路',...",48,"['地铁天通苑北站', '东三旗', '平西王府路口南', '平坊南', '冠雅园东', '...",46
1984,北京658路公交车路线,http://beijing.gongjiao.com/xianlu_47006,嘉铭园,定慧桥南,5:20-22:00,未知,10公里以内票价2元，每增加5公里以内加价1元，最高票价5元,北京公交集团第一客运分公司(服务热线:010-64041170),2015-08-05 01:56:47,"['嘉铭园', '育慧西里', '惠新东桥西', '安慧桥东', '亚运村', '北辰桥西'...",33,"['定慧桥南', '定慧北桥', '五路桥', '营慧寺', '四季青桥南', '四季青桥东...",34
1985,北京323路公交车路线,http://beijing.gongjiao.com/xianlu_47007,富丰桥西,知春里,5:20-23:00,未知,10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价4元,北京公交集团第三客运分公司(服务热线:010-68879184),2015-08-04 06:07:02,"['富丰桥西', '看丹桥', '正阳桥北', '北大地', '北大街', '东大街北口',...",26,"['知春里', '白塔庵', '青云路', '红民村', '四通桥东', '四通桥西', '...",27


以上是所有重复出现过的line_name值，但并不是所有重复的值(例如22路重复出现过，但22路在结果中只有一条，不便于观察除了名字之外是否还有其他字段的重复)。为了找出所有重复的值(例如输出所有22路的记录)，我们可以从原数据中取line_name是这些值的所有行

In [25]:
# 首先定义一个列表,每找出一行line_name在上面范围内的,
# 就将这行加入列表,然后调用concat方法将列表拼接成数据表
dup_lines = []
for name in dup_data.line_name:
    tmp_lines = ori_data[ori_data['line_name'] == name]
    dup_lines.append(tmp_lines)
    dup_data_all = pd.concat(dup_lines)
dup_data_all

,line_name,line_url,line_start,line_stop,line_op_time,line_interval,line_price,line_company,line_up_times,line_station_up,line_station_up_len,line_station_down,line_station_down_len
19,北京22路公交车路线,http://beijing.gongjiao.com/xianlu_22565,牡丹园西,前门,5:00-24:00,未知,无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价3元,北京公交集团电车客运分公司,2015-04-05 03:32:23,"['牡丹园西', '牡丹园', '北太平桥南', '铁狮子坟', '北京师范大学', '小西...",19,"['前门', '前门西', '和平门东', '宣武门东', '西单路口南', '西单商场',...",18
20,北京22路公交车路线,http://beijing.gongjiao.com/xianlu_22566,牡丹园西,前门,5:00-24:00,未知,无人售票线路，单一票制1元，不设找赎。持卡乘车普通卡0.40元/次、学生卡0.20元/次,北京公交集团电车客运分公司,2015-01-29 07:33:25,"['牡丹园西', '牡丹园', '北太平桥南', '铁狮子坟', '北京师范大学', '小西...",19,"['前门', '前门西', '和平门东', '宣武门东', '西单路口南', '西单商场',...",18
278,北京422路公交车路线,http://beijing.gongjiao.com/xianlu_22824,来广营西桥东,安贞桥北,5:30-21:50,未知,无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价3元,北京公交集团第八客运分公司,2015-04-05 03:33:33,"['来广营西桥东', '来广营路口西', '广顺桥南', '利泽中街西口', '望京花园西区...",23,"['安贞桥北', '五路居', '小关', '小关东里', '惠新苑', '惠新里', '对...",23
279,北京422路公交车路线,http://beijing.gongjiao.com/xianlu_22825,三义庙,来广营西桥东,5:30-21:00,未知,12公里以内票价1元，每增加5公里以内加价0.5元。持卡乘车普通卡0.40元/次、学生卡0....,北京公交集团第八客运分公司,2015-01-29 07:33:23,"['三义庙', '四通桥东', '红民村', '大钟寺', '蓟门桥西', '蓟门桥东', ...",29,"['来广营西桥东', '来广营路口西', '利泽中街西口', '望京花园西区', '湖光北街...",29
1106,北京通11路公交车路线,http://beijing.gongjiao.com/xianlu_23653,董村,刘庄,5:40-18:30,未知,NaN,北京恒基客运有限公司,2015-01-29 07:33:14,"['董村', '星光二街', '北神树西口', '北神树小学', '北神树东口', '科创二...",52,"['刘庄', '刘庄北口', '范庄', '结研所', '北京财贸职业学院', '八里桥市场...",52
1704,北京通11路公交车路线,http://beijing.gongjiao.com/xianlu_41275,刘庄,董村,05:40-18:30,未知,最高票价6元,北京恒基客运有限公司,2015-01-29 07:30:32,"['刘庄', '刘庄北口', '范庄', '结研所', '北京财贸职业学院', '八里桥市场...",48,"['董村', '北神树东口', '北神树西口', '北神树小学', '次渠银行', '科创二...",46
1700,北京通45路公交车路线,http://beijing.gongjiao.com/xianlu_40870,通州三中,通州区中医医院,6:00-19:00,未知,10公里以内票价2元，每增加5公里以内加价1元,北京恒基客运有限公司,2015-04-05 04:47:43,"['通州三中', '通州古城村南口', '辛安屯村', '荔景园小区', '玉带河东街路口南...",18,"['通州区中医医院', '翠屏小区东站', '果园环岛南', '地铁九棵树站', '九棵树'...",18
1705,北京通45路公交车路线,http://beijing.gongjiao.com/xianlu_41317,通州三中,通州中医院站,06:00-19:00,10分钟,刷卡优惠成人4折/学生2折,未知,2015-01-29 07:30:32,"['通州三中', '通州古城村南口', '新安屯村', '荔景园小区', '玉带河东街路口南...",19,"['通州中医院站', '翠屏小区东站', '果园环岛南', '城铁九棵树站', '城铁九棵树...",19
1399,北京顺45路公交车路线,http://beijing.gongjiao.com/xianlu_23946,顺义西门,木林,6:28-18:28,未知,10公里以内票价2元，每增加5公里以内加价1元,北京骏马客运有限公司,2015-04-05 03:44:00,"['顺义西门', '物美商城', '顺义火车站', '顺鑫农业', '隆华购物中心', '国...",43,"['木林', '木林镇陈各庄', '蒋各庄北', '蒋各庄', '蒋各庄南', '荣各庄',...",43
1728,北京顺45路公交车路线,http://beijing.gongjiao.com/xianlu_42373,西门,木林,7:29-18:05,未知,NaN,未知,2015-01-29 07:30:23,"['西门', '谊宾商城', '顺义火车站', '顺鑫农业', '隆华', '地铁顺义站',...",38,"['木林', '陈各庄', '蒋各庄', '荣各庄', '后王各庄', '前王各庄', '李...",38


观察dup_data_all，确实同一个线路名字存在重复的记录，而且其余信息也是几乎都相同的，这确认了我们认为的线路”重名“现象是不存在的。但同一条线路的信息具体以哪一个为准呢？注意到有更新时间line_up_time字段，因此我们可以以最新时间的信息为准。

## 剔除重复数据

这里考虑两种思路。第一种，直接对原数据进行操作，当line_name存在重复时，保留最近更新时间的记录。第二种，将原数据中的dup_data_all部分完全删除，拼接上dup_data_all去除重复的部分。两种思路都需要删除line_name重复的记录，保留一个时间最新的。pandas本身有drop_duplicates方法，使用keep=last或keep=first参数就可以指定保留的记录。但在这之前我们需要将line_up_time转换为pandas可以识别的时间类型，然后对其进行排序。

In [26]:
# 方法一
# 使用to_datetime方法，指定format，将字符串转换为pandas的时间类型
ori_data['line_up_times'] = pd.to_datetime(
    ori_data['line_up_times'], format='%Y-%m-%d %H:%M:%S')
#使用sort_values方法，对line_name和line_up_time排序
ori_data.sort_values(
    by=['line_name', 'line_up_times'], ascending=[True, True], inplace=True)
#由于是升序排列，所以keep=last就可以保留最新事件的记录
drop_dup_line1 = ori_data.drop_duplicates(subset=['line_name'], keep='last')
len(drop_dup_line1)

1986

In [27]:
# 方法二
#使用to_datetime方法，指定format，将字符串转换为pandas的时间类型
dup_data_all['line_up_times'] = pd.to_datetime(
    dup_data_all['line_up_times'], format='%Y-%m-%d %H:%M:%S')
#使用sort_values方法，对line_name和line_up_time排序
ori_data.sort_values(
    by=['line_name', 'line_up_times'], ascending=[True, True], inplace=True)
#使用keep=last保留时间更新的记录
dup_data_all.drop_duplicates(subset=['line_name'], keep='last', inplace=True)

#获取原数据中剔除了重复线路的数据：取名字不在dup_data_all的line_name集合中的记录
other_data = ori_data[~ori_data['line_name'].isin(dup_data_all.line_name)]
# 拼接两部分数据
drop_dup_line2 = pd.concat([other_data, dup_data_all])
len(drop_dup_line2)

1986

## 剔除干扰数据

虽然我们爬取的是公交路线，但程序运行过程中我也发现了地铁的线路(其实地铁也是广义上的公交啦)。如果我们的目的是对纯粹的公交线路进行分析，就需要将地铁的线路删除。直观的思路是剔除线路名称中含有“地铁”的记录。

In [28]:
# 使用.str将其转换为字符串就可以使用字符串的contains方法
is_subway = drop_dup_line1.line_name.str.contains('地铁')
subway_data = drop_dup_line1[is_subway]
subway_data

,line_name,line_url,line_start,line_stop,line_op_time,line_interval,line_price,line_company,line_up_times,line_station_up,line_station_up_len,line_station_down,line_station_down_len
946,北京地铁10号线内环(M10),http://beijing.gongjiao.com/xianlu_23493,苏州街,巴沟,4:54-22:08,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:13,"['苏州街', '海淀黄庄', '知春里', '知春路', '西土城', '牡丹园', '健...",45,"['地铁郭公庄站', '地铁丰台科技园站', '科怡路', '丰台南路', '丰台东大街',...",13
945,北京地铁10号线外环(M10),http://beijing.gongjiao.com/xianlu_23492,火器营,巴沟,4:49-22:35,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:13,"['火器营', '长春桥', '车道沟', '地铁慈寿寺站', '西钓鱼台', '公主坟',...",45,"['地铁郭公庄站', '地铁丰台科技园站', '科怡路', '丰台南路', '丰台东大街',...",13
947,北京地铁13号线(M13),http://beijing.gongjiao.com/xianlu_23494,西直门,东直门,5:35-22:42,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:14,"['西直门', '地铁大钟寺站', '知春路', '地铁五道口站', '地铁上地站', '地...",16,"['东直门', '地铁柳芳站', '光熙门', '地铁芍药居站', '地铁望京西站', '地...",16
948,北京地铁14号线(M14),http://beijing.gongjiao.com/xianlu_23495,张郭庄,西局,5:30-22:10,未知,单一票价2元,北京京港地铁有限公司,2015-01-29 07:33:15,"['张郭庄', '园博园', '大瓦窑', '郭庄子', '大井', '七里庄', '西局']",7,"['西局', '七里庄', '大井', '郭庄子', '大瓦窑', '园博园', '张郭庄']",7
1844,北京地铁14号线东段(M14),http://beijing.gongjiao.com/xianlu_45377,金台路,地铁善各庄站,6:00-23:00,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京京港地铁有限公司,2015-04-05 04:51:13,"['金台路', '枣营', '东风北桥', '将台', '望京南', '阜通', '地铁望京...",10,"['地铁善各庄站', '来广营', '东湖渠', '地铁望京站', '阜通', '望京南',...",10
1786,北京地铁14号线西段(M14),http://beijing.gongjiao.com/xianlu_44780,地铁张郭庄站,地铁西局站,5:30-22:10,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京京港地铁有限公司,2015-04-05 03:41:14,"['地铁张郭庄站', '地铁园博园站', '地铁大瓦窑站', '地铁郭庄子站', '大井',...",7,"['地铁西局站', '七里庄', '大井', '地铁郭庄子站', '地铁大瓦窑站', '地铁...",7
949,北京地铁15号线(M15),http://beijing.gongjiao.com/xianlu_23496,地铁俸伯站,清华东路西口,5:30-22:11,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营四分公司,2015-04-05 03:41:14,"['地铁俸伯站', '地铁顺义站', '地铁石门站', '地铁南法信站', '地铁后沙峪站'...",18,"['清华东路西口', '六道口', '北沙滩', '奥林匹克公园', '安立路', '关庄'...",18
935,北京地铁1号线(M1),http://beijing.gongjiao.com/xianlu_23482,地铁苹果园站(0.000),四惠东(30.440),5:10-22:55,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营二分公司,2015-04-05 03:41:10,"['地铁苹果园站(0.000)', '古城(2.606)', '八角游乐园(4.527)',...",23,"['四惠东(30.440)', '四惠(28.726)', '大望路(27.053)', '...",23
937,北京地铁2号线内环(M2),http://beijing.gongjiao.com/xianlu_23484,鼓楼大街,积水潭,5:03-21:55,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:11,"['鼓楼大街', '安定门', '雍和宫', '东直门', '东四十条', '朝阳门', '...",18,"['3号航站楼', '2号航站楼', '三元桥(地铁)', '东直门']",4
939,北京地铁2号线外环(M2),http://beijing.gongjiao.com/xianlu_23486,车公庄,西直门,5:09-22:14,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:11,"['车公庄', '阜成门', '复兴门', '长椿街', '宣武门', '和平门', '前门...",18,"['地铁次渠站', '次渠南', '地铁经海路站', '地铁同济南路站', '地铁荣昌东街站...",13


从上图左侧可以看到subway_data的结果不仅仅有地铁，还有一些地铁有关的通勤线路，其实是公交。因此不能直接删除line_name中含有“地铁”的记录，我们使用line_conpany中含有“地铁”来区分，效果更好。

In [30]:
is_subway2 = drop_dup_line1.line_company.str.contains('地铁')
subway_data2 = drop_dup_line1[is_subway2]
subway_data2

,line_name,line_url,line_start,line_stop,line_op_time,line_interval,line_price,line_company,line_up_times,line_station_up,line_station_up_len,line_station_down,line_station_down_len
946,北京地铁10号线内环(M10),http://beijing.gongjiao.com/xianlu_23493,苏州街,巴沟,4:54-22:08,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:13,"['苏州街', '海淀黄庄', '知春里', '知春路', '西土城', '牡丹园', '健...",45,"['地铁郭公庄站', '地铁丰台科技园站', '科怡路', '丰台南路', '丰台东大街',...",13
945,北京地铁10号线外环(M10),http://beijing.gongjiao.com/xianlu_23492,火器营,巴沟,4:49-22:35,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:13,"['火器营', '长春桥', '车道沟', '地铁慈寿寺站', '西钓鱼台', '公主坟',...",45,"['地铁郭公庄站', '地铁丰台科技园站', '科怡路', '丰台南路', '丰台东大街',...",13
947,北京地铁13号线(M13),http://beijing.gongjiao.com/xianlu_23494,西直门,东直门,5:35-22:42,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:14,"['西直门', '地铁大钟寺站', '知春路', '地铁五道口站', '地铁上地站', '地...",16,"['东直门', '地铁柳芳站', '光熙门', '地铁芍药居站', '地铁望京西站', '地...",16
948,北京地铁14号线(M14),http://beijing.gongjiao.com/xianlu_23495,张郭庄,西局,5:30-22:10,未知,单一票价2元,北京京港地铁有限公司,2015-01-29 07:33:15,"['张郭庄', '园博园', '大瓦窑', '郭庄子', '大井', '七里庄', '西局']",7,"['西局', '七里庄', '大井', '郭庄子', '大瓦窑', '园博园', '张郭庄']",7
1844,北京地铁14号线东段(M14),http://beijing.gongjiao.com/xianlu_45377,金台路,地铁善各庄站,6:00-23:00,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京京港地铁有限公司,2015-04-05 04:51:13,"['金台路', '枣营', '东风北桥', '将台', '望京南', '阜通', '地铁望京...",10,"['地铁善各庄站', '来广营', '东湖渠', '地铁望京站', '阜通', '望京南',...",10
1786,北京地铁14号线西段(M14),http://beijing.gongjiao.com/xianlu_44780,地铁张郭庄站,地铁西局站,5:30-22:10,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京京港地铁有限公司,2015-04-05 03:41:14,"['地铁张郭庄站', '地铁园博园站', '地铁大瓦窑站', '地铁郭庄子站', '大井',...",7,"['地铁西局站', '七里庄', '大井', '地铁郭庄子站', '地铁大瓦窑站', '地铁...",7
949,北京地铁15号线(M15),http://beijing.gongjiao.com/xianlu_23496,地铁俸伯站,清华东路西口,5:30-22:11,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营四分公司,2015-04-05 03:41:14,"['地铁俸伯站', '地铁顺义站', '地铁石门站', '地铁南法信站', '地铁后沙峪站'...",18,"['清华东路西口', '六道口', '北沙滩', '奥林匹克公园', '安立路', '关庄'...",18
935,北京地铁1号线(M1),http://beijing.gongjiao.com/xianlu_23482,地铁苹果园站(0.000),四惠东(30.440),5:10-22:55,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营二分公司,2015-04-05 03:41:10,"['地铁苹果园站(0.000)', '古城(2.606)', '八角游乐园(4.527)',...",23,"['四惠东(30.440)', '四惠(28.726)', '大望路(27.053)', '...",23
937,北京地铁2号线内环(M2),http://beijing.gongjiao.com/xianlu_23484,鼓楼大街,积水潭,5:03-21:55,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:11,"['鼓楼大街', '安定门', '雍和宫', '东直门', '东四十条', '朝阳门', '...",18,"['3号航站楼', '2号航站楼', '三元桥(地铁)', '东直门']",4
939,北京地铁2号线外环(M2),http://beijing.gongjiao.com/xianlu_23486,车公庄,西直门,5:09-22:14,未知,6公里以内票价3元，12公里以内票价4元，22公里以内票价5元，32公里以内票价6元，32公...,北京市地铁运营有限公司运营三分公司,2015-04-05 03:41:11,"['车公庄', '阜成门', '复兴门', '长椿街', '宣武门', '和平门', '前门...",18,"['地铁次渠站', '次渠南', '地铁经海路站', '地铁同济南路站', '地铁荣昌东街站...",13


In [31]:
# 在drop_dup_line1的基础上，筛选出线路名称不在subway_data2中的线路名称的记录即可
clean_data = drop_dup_line1[~drop_dup_line1['line_name'].isin(subway_data2.
                                                              line_name)]
len(clean_data)

1963

In [35]:
#通过是否含有“公交车线路”进行筛选，结果是1955，应该就是少了那些“接驳线”
clean_data3 = drop_dup_line1[drop_dup_line1.line_name.str.contains("公交车路线")]
len(clean_data3)

1955

In [36]:
#找出在clean_data的line_name中但是不在clean_data3的line_name中的数据
list(
    set(clean_data.line_name.values).difference(
        set(clean_data3.line_name.values)))

['北京大兴清河园-地铁旧宫站接驳线',
 '北京城际通勤班车[潮白河孔雀城-地铁潞城站](跨省)',
 '北京大兴国际企业大道-地铁经海路站接驳线',
 '北京通勤班车[里二泗总站-地铁九棵树站]',
 '北京西红门地铁-公交驾校快捷班车',
 '北京大兴北工大软件园-地铁荣京东街站接驳线',
 '北京大兴马赛公馆-地铁枣园站接驳线',
 '北京大兴兴康家园-地铁枣园站接驳线',
 '北京大兴数字工厂-地铁经海路站接驳线']

In [37]:
#找出在clean_data3的line_name中但是不在clean_data的line_name中的数据
list(
    set(clean_data3.line_name.values).difference(
        set(clean_data.line_name.values)))

['北京机场快轨(L1)公交车路线']

## 处理缺失值

可以使用isnull().sum()方法查看。发现票价有230个缺失值。参见后面的图片。对于缺失值我们需要在预处理阶段对其进行填充。考虑到票价数据本身不是纯粹的价格数据，而是一大串的文字描述，并且在公交的这种场景下，其实不同线路的票价差别不是很大，因此我们可以使用众数对缺失值进行填充。使用mode方法查看众数，使用fillna方法填补缺失值。

In [38]:
#使用mode()方法查看line_price的众数
clean_data.line_price.mode()

0    10公里以内票价2元，每增加5公里以内加价1元
dtype: object

In [40]:
#使用value_counts()方法查看每一个取值出现的次数，第一个也是众数
clean_data.line_price.value_counts()

10公里以内票价2元，每增加5公里以内加价1元                                                                367
无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价3元                                             194
无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价2元                                             163
无人售票线路，单一票制1元，不设找赎。持卡乘车普通卡0.40元/次、学生卡0.20元/次                                           117
无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价4元                                             104
10公里以内票价2元，每增加5公里以内加价1元，最高票价5元                                                          85
无人售票线路，10公里以内票价2元，每增加5公里以内加价1元                                                          80
10公里以内票价2元，每增加5公里以内加价1元，最高票价6元                                                          67
10公里以内票价2元，每增加5公里以内加价1元，最高票价4元                                                          49
无人售票线路，10公里以内票价2元，每增加5公里以内加价1元，不设找赎，最高票价5元                                              46
10公里以内票价2元，每增加5公里以内加价1元，最高票价7元                                                          38

In [43]:
# 用众数进行填充
clean_data.line_price.fillna(clean_data.line_price.mode()[0], inplace=True)
clean_data.isnull().sum()

line_name                0
line_url                 0
line_start               0
line_stop                0
line_op_time             0
line_interval            0
line_price               0
line_company             0
line_up_times            0
line_station_up          0
line_station_up_len      0
line_station_down        0
line_station_down_len    0
dtype: int64